# Countries Explorative Data Analysis (EDA)

## Overview
The data contains information about all the 195 countries in the world.
The data can help analyze the population, area, and different details about the countries

## Questions and Analysis
1. What is the population distribution?
2. What is the most and least populated country?
3. What is the most and least populated continent?
4. What is the country with the largest and smallest average area?
5. What is the continent with the largest and smallest average area?
6. What is the country with the most inequality of wealth?
7. What is the continent with the most/least equality of wealth?
8. What is the most spoken language in the world?
9. How many countries drive on the left side?
10. How many countries are still not independant?
11. What is the continent with the most countries?
12. What country got his independence last?